In [1]:
import numpy as np
from tensorflow import keras
import json
import pickle
import random
import nltk
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
lemmatizer = WordNetLemmatizer()

In [3]:
intents = json.loads(open('intents.json').read())
words = []
classes = []
documents = []
ignore_letters = ['?','!','.', ',']

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(words)

['Hi', 'khoe', 'khong', 'em', 'oi', 'ban', 'khoe', 'khong', 'Hello', 'Good', 'day', 'Whats', 'up', 'xin', 'chao', 'cya', 'See', 'you', 'later', 'Goodbye', 'I', 'am', 'Leaving', 'Have', 'a', 'Good', 'day', 'bye', 'tam', 'biet', 'how', 'old', 'how', 'old', 'is', 'tim', 'what', 'is', 'your', 'age', 'how', 'old', 'are', 'you', 'age', '?', 'tuoi', 'tuoi', 'cua', 'toi', 'what', 'is', 'your', 'name', 'what', 'should', 'I', 'call', 'you', 'whats', 'your', 'name', '?', 'ten', 'ban', 'ten', 'gi', 'Id', 'like', 'to', 'buy', 'something', 'whats', 'on', 'the', 'menu', 'what', 'do', 'you', 'reccommend', '?', 'could', 'i', 'get', 'something', 'to', 'eat', 'ban', 'hoc', 'truong', 'nao', 'truong', 'ten', 'truong', 'hoc', 'o', 'dau', 'where', 'o', 'dau', 'ban', 'o', 'dau', 'khuc', 'nao', 'dia', 'chi', 'truong', 'truong', 'o', 'cho', 'nao', 'truong', 'o', 'dau', 'help', 'giup', 'toi', 'voi', 'ban', 'co', 'the', 'giup', 'toi', 'duoc', 'khong']


In [5]:
#lemmatize và hạ thấp từng từ và loại bỏ các từ trùng lặp
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
# sắp xếp class
words = sorted(set(words))
classes = sorted(set(classes))
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))
#tạo training data
training =[]
#tạo mảng trống cho đầu ra
output_empty = [0]*len(classes)
#training set, bag of words for every sentence
for document in documents:
    #khởi tạo bag of words:
    bag=[]
    #danh sách các từ được mã hóa cho patterns
    word_patterns = document[0]
    #lemmatize từng từ - tạo từ cơ bản, cố gắng biểu diễn các từ có liên quan
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    #đầu ra là '0' cho mỗi tag và 1 cho tag hiện tại (cho mỗi pattern)
    output_row = list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag,output_row])

In [10]:
#xáo cái training
random.shuffle(training)
dtype=object
training = np.array(training)
#create training and testting list. X- patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [7]:
model=keras.Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
sgd = tf.keras.optimizers.legacy.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer= sgd, metrics= ['accuracy'])
hist=model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodel.h5',hist)
print("training is done")

Epoch 1/200
9/9 [==============================] - 1s 5ms/step - loss: 2.3581 - accuracy: 0.1111
Epoch 2/200
9/9 [==============================] - 0s 8ms/step - loss: 2.2920 - accuracy: 0.1111
Epoch 3/200
9/9 [==============================] - 0s 6ms/step - loss: 2.2108 - accuracy: 0.1778
Epoch 4/200
9/9 [==============================] - 0s 4ms/step - loss: 2.1390 - accuracy: 0.3333
Epoch 5/200
9/9 [==============================] - 0s 8ms/step - loss: 2.0269 - accuracy: 0.3111
Epoch 6/200
9/9 [==============================] - 0s 6ms/step - loss: 2.0548 - accuracy: 0.2889
Epoch 7/200
9/9 [==============================] - 0s 4ms/step - loss: 1.9878 - accuracy: 0.2889
Epoch 8/200
9/9 [==============================] - 0s 6ms/step - loss: 1.9051 - accuracy: 0.3778
Epoch 9/200
9/9 [==============================] - 0s 5ms/step - loss: 1.7962 - accuracy: 0.5111
Epoch 10/200
9/9 [==============================] - 0s 6ms/step - loss: 1.6903 - accuracy: 0.3778
Epoch 11/200
9/9 [===========

9/9 [==============================] - 0s 2ms/step - loss: 0.1043 - accuracy: 0.9778
Epoch 85/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0772 - accuracy: 0.9778
Epoch 86/200
9/9 [==============================] - 0s 2ms/step - loss: 0.1636 - accuracy: 0.9556
Epoch 87/200
9/9 [==============================] - 0s 2ms/step - loss: 0.1620 - accuracy: 0.9556
Epoch 88/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0807 - accuracy: 0.9778
Epoch 89/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0931 - accuracy: 0.9778
Epoch 90/200
9/9 [==============================] - 0s 3ms/step - loss: 0.1896 - accuracy: 0.9333
Epoch 91/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0915 - accuracy: 0.9778
Epoch 92/200
9/9 [==============================] - 0s 2ms/step - loss: 0.1180 - accuracy: 0.9778
Epoch 93/200
9/9 [==============================] - 0s 3ms/step - loss: 0.1504 - accuracy: 0.9556
Epoch 94/200
9/9 [===============

9/9 [==============================] - 0s 3ms/step - loss: 0.0866 - accuracy: 0.9778
Epoch 168/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0360 - accuracy: 1.0000
Epoch 169/200
9/9 [==============================] - 0s 4ms/step - loss: 0.0510 - accuracy: 1.0000
Epoch 170/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0331 - accuracy: 1.0000
Epoch 171/200
9/9 [==============================] - 0s 2ms/step - loss: 0.1398 - accuracy: 0.9556
Epoch 172/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0627 - accuracy: 1.0000
Epoch 173/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0488 - accuracy: 1.0000
Epoch 174/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0317 - accuracy: 1.0000
Epoch 175/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0411 - accuracy: 0.9778
Epoch 176/200
9/9 [==============================] - 0s 2ms/step - loss: 0.0554 - accuracy: 1.0000
Epoch 177/200
9/9 [=====